# 🤝 Lab 04: Multi-Agent Systems

**Goal:** Understand when and how to use multiple agents collaborating on a task.

**Time:** 60 minutes

---

## What you'll learn:
- When to use multi-agent vs single agent
- Orchestration vs choreography
- Manager-Worker pattern

---

In [ ]:
#@title 🔧 Setup
!pip install -q google-generativeai smolagents litellm

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from smolagents import CodeAgent, tool, ManagedAgent
from smolagents.models import LiteLLMModel

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)

print("✅ Setup OK")

## Part 1: When Multi-Agent?

### Single Agent is enough when:
- Task is straightforward
- All tools relate to one domain
- No specialization needed

### Multi-Agent when:
- Task has multiple independent parts
- Different parts require different expertise
- You want to parallelize work
- You need review and control

## Part 2: Manager-Worker Pattern

```
┌──────────────────┐
│  MANAGER AGENT   │ ← Decides, delegates, combines
└────────┬─────────┘
         │
    ┌────┴────┬────────────┐
    │         │            │
┌───┴───┐ ┌───┴───┐ ┌──────┴──────┐
│Worker │ │Worker │ │   Worker    │
│  A    │ │  B    │ │     C       │
└───────┘ └───────┘ └─────────────┘
Research   Writer    Fact-checker
```

In [ ]:
#@title 👷 Worker Agents: Specialized agents

# Worker 1: Research Agent
@tool
def search_web(query: str) -> str:
    """Searches for information on the web (simulated)."""
    # Demo data
    data = {
        "ai trends 2026": "Top AI trends 2026: 1) Agentic AI, 2) Multimodal models, 3) Edge AI, 4) AI governance",
        "python popularity": "Python remains #1 language for AI/ML with 30% popularity growth.",
        "llm costs": "LLM inference costs dropped by 90% over the last 2 years."
    }
    for key, value in data.items():
        if key in query.lower():
            return value
    return "No results for: " + query

research_agent = CodeAgent(
    tools=[search_web],
    model=model,
    max_steps=3,
    name="research_agent",
    description="Specialist in information search and research."
)

print("✅ Research Agent created")

In [ ]:
#@title 👷 Worker 2: Writer Agent

@tool
def format_as_bullet_points(text: str) -> str:
    """Reformats text into bullet points."""
    lines = text.split('.')
    bullets = "\n".join([f"• {line.strip()}" for line in lines if line.strip()])
    return bullets

@tool
def create_summary(text: str, max_words: int = 50) -> str:
    """Creates a brief summary of the text."""
    words = text.split()
    if len(words) <= max_words:
        return text
    return ' '.join(words[:max_words]) + '...'

writer_agent = CodeAgent(
    tools=[format_as_bullet_points, create_summary],
    model=model,
    max_steps=3,
    name="writer_agent",
    description="Specialist in writing and formatting text."
)

print("✅ Writer Agent created")

In [ ]:
#@title 👔 Manager Agent

# Wrap workers as ManagedAgents
managed_research = ManagedAgent(
    agent=research_agent,
    name="researcher",
    description="Searches and retrieves information on a given topic."
)

managed_writer = ManagedAgent(
    agent=writer_agent,
    name="writer",
    description="Writes and formats text based on information."
)

# Manager agent - orchestrates others
manager_agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=10,
    managed_agents=[managed_research, managed_writer],
    system_prompt="""
    You are a team manager. You have available:
    - researcher: searches for information
    - writer: writes and formats text
    
    For each task:
    1. First use researcher to gather information
    2. Then use writer to create the final output
    3. Combine results into a coherent response
    """
)

print("✅ Manager Agent created")
print("   Manages: researcher, writer")

In [ ]:
#@title ▶️ Test Multi-Agent system

task = "Prepare a brief overview of AI trends for 2026 in bullet point format."

print(f"📋 Task: {task}")
print("=" * 50)

result = manager_agent.run(task)

print("=" * 50)
print(f"\n✅ Result:\n{result}")

## Part 3: Practical Example - Content Pipeline

In [ ]:
#@title 📰 Content Creation Pipeline

# Simulated article database
ARTICLES_DB = []

# --- RESEARCH AGENT ---
@tool
def gather_topic_info(topic: str) -> str:
    """Gathers information about a topic."""
    info = {
        "ai": "AI is transforming industry. Key areas: automation, prediction, personalization.",
        "cloud": "Cloud computing grows 20% annually. Main players: AWS, Azure, GCP.",
        "security": "Cyber attacks increased by 40%. Zero trust is the new standard."
    }
    for key, value in info.items():
        if key in topic.lower():
            return f"Research for '{topic}':\n{value}"
    return f"Topic '{topic}' requires additional research."

researcher = CodeAgent(
    tools=[gather_topic_info],
    model=model,
    max_steps=3,
    name="researcher",
    description="Does research on topics"
)

# --- WRITER AGENT ---
@tool
def write_article(title: str, content: str) -> str:
    """Writes an article with the given title and content."""
    article = f"""
# {title}

{content}

---
*Generated by AI Academy*
    """
    return article.strip()

writer = CodeAgent(
    tools=[write_article],
    model=model,
    max_steps=3,
    name="writer",
    description="Writes articles"
)

# --- EDITOR AGENT ---
@tool
def check_quality(text: str) -> str:
    """Checks text quality."""
    issues = []
    
    if len(text) < 100:
        issues.append("Text is too short")
    if text.count('#') == 0:
        issues.append("Missing heading")
    if '.' not in text:
        issues.append("Missing punctuation")
    
    if issues:
        return "❌ Issues: " + ", ".join(issues)
    return "✅ Quality OK"

@tool
def publish_article(article: str) -> str:
    """Publishes article to database."""
    ARTICLES_DB.append(article)
    return f"✅ Article published! (total {len(ARTICLES_DB)} articles)"

editor = CodeAgent(
    tools=[check_quality, publish_article],
    model=model,
    max_steps=3,
    name="editor",
    description="Reviews and publishes articles"
)

print("✅ Content Pipeline agents created")

In [ ]:
#@title 👔 Content Manager

content_manager = CodeAgent(
    tools=[],
    model=model,
    max_steps=15,
    managed_agents=[
        ManagedAgent(researcher, "researcher", "Gets info about topic"),
        ManagedAgent(writer, "writer", "Writes article"),
        ManagedAgent(editor, "editor", "Reviews and publishes")
    ],
    system_prompt="""
    You are a content manager. Article creation process:
    
    1. RESEARCH: Use 'researcher' to get info about the topic
    2. WRITE: Use 'writer' to write the article
    3. REVIEW: Use 'editor' to check quality
    4. PUBLISH: If quality is OK, use 'editor' to publish
    
    If editor finds issues, go back to writer.
    """
)

print("✅ Content Manager created")

In [ ]:
#@title ▶️ Create an article

result = content_manager.run("""
Create a short article about AI in enterprise environments.
The article should be informative and professional.
""")

print("=" * 50)
print("RESULT:")
print("=" * 50)
print(result)

## Part 4: Patterns and Anti-Patterns

### ✅ Good patterns:

1. **Manager-Worker** - one agent coordinates specialists
2. **Pipeline** - sequential processing (research → write → review)
3. **Peer Review** - agents review each other's work

### ❌ Anti-patterns:

1. **Too many agents** - unnecessary complexity
2. **Circular dependencies** - agents call each other endlessly
3. **No clear ownership** - nobody is responsible for the result

In [ ]:
#@title ❌ Anti-pattern: Too many agents

# This is a BAD example - we don't need 5 agents for a simple task!

# Agent 1: Question reader
# Agent 2: Context analyzer  
# Agent 3: Searcher
# Agent 4: Answer formulator
# Agent 5: Grammar checker

# Better: One agent with good tools!

print("❌ Don't use 5 agents when 1 with good tools is enough!")

---

## 🏋️ Exercise: Customer Service Multi-Agent

**Task:** Create a multi-agent system for customer service:

1. **Triage Agent** - categorizes requests (billing, technical, general)
2. **Billing Agent** - handles billing questions
3. **Technical Agent** - handles technical issues
4. **Manager Agent** - coordinates and escalates

In [ ]:
#@title ✏️ Your solution

# TODO: Implement agents

# Triage tools
@tool
def categorize_request(request: str) -> str:
    """Categorizes a customer request."""
    # TODO: Implement
    pass

# Billing tools
@tool
def check_invoice(invoice_id: str) -> str:
    """Checks an invoice."""
    # TODO: Implement
    pass

# Technical tools
@tool
def diagnose_issue(description: str) -> str:
    """Diagnoses a technical issue."""
    # TODO: Implement
    pass

# TODO: Create agents and manager agent

# Test:
# print(manager.run("My invoice #12345 is incorrect"))
# print(manager.run("The app crashes when I log in"))

---

## 📝 Reflection

1. **When is multi-agent better than single agent?**

2. **What are the disadvantages of multi-agent systems?** (Hint: latency, cost, complexity)

3. **How would you design a multi-agent system for your real project?**

---

## 🎓 Congratulations!

You've completed the core AI Academy labs!

### What you learned:
- ✅ AI agent basics (Lab 01)
- ✅ Tools for agents (Lab 02)  
- ✅ RAG pattern (Lab 03)
- ✅ Multi-agent systems (Lab 04)

### Next steps:
- Deployment to HuggingFace Spaces
- Team project
- Hackathon!